In [69]:
# Import necessary libraries
import numpy as np
import math
import re
import sys
import pyspark
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession, HiveContext
import pyspark.sql.functions as f
from pyspark.sql.window import Window
import pandas as pd


#sc1 = sp.SparkContext.getOrCreate()
sc = SparkContext("local", "First-app")

In [70]:
# Create a SparkSession and sqlContext
spark = SparkSession.builder.master(sc.master) \
                    .appName(sc.appName) \
                    .enableHiveSupport() \
                    .getOrCreate()

sqlContext = SQLContext(sc)

In [137]:
# A function for text formatting
#specialCharacters = "!@#$%^&'*()+?_=<>1234567890/"
special_char_remove = ["!","@","#","$","%","^","&","'","*","“","”","—","’","‘","·","´","(",")","/","+","ð","°","º"
                       ,"?","•","~","£","_","|","=","<",">","æ","Æ","œ","½","―","…","1","2","3","4","5","6","7","8","9","0"
                       ,"è","à","ê","â","ù","ô","ö","Ö","é","î","û","ì","ü","Ü","ò","á","ñ","ç","ó","ä","ë","ï","í","ú","ý"];

#I thought you would have gone home long ago. Dinner ’ll be 
def FilterAndFormatText(inputText):
    arrayWords = []
    returnText = re.sub("[_.;:\[\]\"?!,(){}-]", " ", inputText)
    arrayWords = GetFilteredText(returnText.split(' '))
    #arrayWords = FilterNumberTexts(arrayWords)
    #arrayWords = FilterSpecialCharacters(arrayWords)
    finalTextFiltered = " ".join(arrayWords)
    #return re.sub('[^a-zA-Z ]+', '', finalTextFiltered).lower()
    return re.sub('[^a-zA-Z ]+', '', finalTextFiltered).lower()

def GetFilteredText(arrInput):
    filteredArray = []
    for i in range(len(arrInput)):
        print(arrInput[i].encode('utf-8'))
        if any(s in arrInput[i].lower() for s in special_char_remove):
            print(arrInput[i])
        else:
            filteredArray.append(arrInput[i])
           
    return filteredArray

#print(FilterAndFormatText("Kubernetes is an open-source system for automating deployment, scaling, and management of containerized applications."
#+" Kubernetes KuberneteS Kuber5netes kubernetes Kubernetes7 KuberneTes "
#+" SystEM Sys System. system, sYstem> >>> 8765 Kubernetes@Kubernetes.com applications. applications (applications,applications)++({applications})"
#+" {PPP} {nnb}"
#+" gg hh jj k mmm iii yyy vv ccc ff rr eee ss dd rrr tttt yy tt tt ttt uu iii www qqq aa ww33 ee ddd cc vvb nn"))

In [138]:
# Create a function to accept the file and generate the output the file according to input
def ParseFileData(fileName):
    filePath = "D:\Project\Input Output\\"+fileName+".txt"
    textFile = sc.textFile(filePath)

    # WordCount started
    textFlatWord = textFile.flatMap(lambda line: FilterAndFormatText(line).split(' ')) \
                            .map(lambda word: (word, 1)) \
                           .reduceByKey(lambda x, y: x + y)

    # CharCount started
    textFlatChar = textFile.flatMap(lambda each: FilterAndFormatText(each)) \
                            .map(lambda char: char) \
                            .map(lambda c: (c, 1)) \
                            .reduceByKey(lambda v1, v2: v1 + v2)

    # Convert flat data int DataFrame
    wordDF = spark.createDataFrame(textFlatWord,['Word', 'Frequency'])

    # Convert flat data int DataFrame
    charDF = spark.createDataFrame(textFlatChar,['Letter', 'Frequency'])
    
    #word Calculation
    wordDF = wordDF[wordDF["Word"] != '']
    wordDF = wordDF[wordDF["Word"] != ' ']
    windowSpec = Window.orderBy(f.desc("Frequency"), f.asc("Word"))
    wordDF = wordDF.withColumn("Rank",f.row_number().over(windowSpec))
    
    totalRank = wordDF.select("Rank").rdd.max()[0]
    
    T1 = math.ceil((5 * totalRank)/100)
    T2 = math.floor((47.5 * totalRank)/100)
    T3 = math.ceil((52.5 * totalRank)/100)
    T4 = math.floor((95 * totalRank)/100)
    
    dfPopular = wordDF[wordDF["Rank"].between(1, T1)]
    dfCommon = wordDF[wordDF["Rank"].between(T2, T3)]
    dfRare = wordDF[wordDF["Rank"] >= T4]
    
    charDF = charDF[charDF["Letter"] != ' ']
    charDF = charDF[charDF["Letter"] != '']

    charWindowSpec = Window.orderBy(f.desc("Frequency"), f.asc("Letter"))
    charDF = charDF.withColumn("Rank",f.row_number().over(charWindowSpec))
    
    charTotalRank = charDF.select("Rank").rdd.max()[0]
    
    T1Char = math.ceil((5 * charTotalRank)/100)
    T2Char = math.floor((47.5 * charTotalRank)/100)
    T3Char = math.ceil((52.5 * charTotalRank)/100)
    T4Char = math.floor((95 * charTotalRank)/100)
    
    dfPopularChar = charDF[charDF["Rank"].between(1, T1Char)]
    dfCommonChar = charDF[charDF["Rank"].between(T2Char, T3Char)]
    dfRareChar = charDF[charDF["Rank"] >= T4Char]
    
    file1 = open("output-"+fileName + ".txt","w")

    file1.write('-----------------------------------------------------------------------------------------\n')
    file1.write('Output for  ' + fileName + '\n')
    file1.write('-----------------------------------------------------------------------------------------\n')
    
    file1.write('total number of distinct words= ' + str(totalRank) + '\n')
    file1.write('popular_threshold_word= ' + str(T1) + '\n')
    file1.write('common_threshold_l_word= ' + str(T2) + '\n')
    file1.write('common_threshold_u_word= ' + str(T3) + '\n')
    file1.write('rare_threshold_word= ' + str(T4) + '\n')
    
    def WriteToFileDFWord(df, writeType):
        file1.write('\n' + writeType + '\n')
        file1.write('+------+---------+------+\n')
        file1.write('|Rank|Frequency|Word  |\n')
        file1.write('+------+---------+------+\n')

        pandasDF = df.toPandas()
        for index, row in pandasDF.iterrows():
            #file1.write('| ' + str(row['Letter']) + '    | ' + str(row['Frequency']) + '       | ' + str(row['Rank']) + '   |\n')
            file1.write(str(row['Rank']) +'    '+ str(row['Frequency']) +'    '+ str(row['Word']) + '\n')

        file1.write('+------+---------+------+\n')
    
    WriteToFileDFWord(dfPopular, 'Popular Words')
    WriteToFileDFWord(dfCommon, 'Common Word')
    WriteToFileDFWord(dfRare, 'Rare Word')
    
    
    file1.write('\n-----------------------------------------------------------------------------------------\n')
    file1.write('total number of distinct letters= ' + str(charTotalRank) + '\n')
    file1.write('popular_threshold_letter= ' + str(T1Char) + '\n')
    file1.write('common_threshold_l_letter= ' + str(T2Char) + '\n')
    file1.write('common_threshold_u_letter= ' + str(T3Char) + '\n')
    file1.write('rare_threshold_letter= ' + str(T4Char) + '\n')
    file1.write('-----------------------------------------------------------------------------------------\n')

    def WriteToFileDF(df, writeType):
        file1.write('\n' + writeType + '\n')
        file1.write('+------+---------+------+\n')
        file1.write('|Rank|Frequency|Letter  |\n')
        file1.write('+------+---------+------+\n')

        pandasDF = df.toPandas()
        for index, row in pandasDF.iterrows():
            #file1.write('| ' + str(row['Letter']) + '    | ' + str(row['Frequency']) + '       | ' + str(row['Rank']) + '   |\n')
            file1.write(str(row['Rank']) +'    '+ str(row['Frequency']) +'    '+ str(row['Letter']) + '\n')

        file1.write('+------+---------+------+\n')

    WriteToFileDF(dfPopularChar, 'Popular Letters')
    WriteToFileDF(dfCommonChar, 'Common Letters')
    WriteToFileDF(dfRareChar, 'Rare Letters')
    file1.close()


In [140]:
ParseFileData('sample-a')
ParseFileData('sample-b')
ParseFileData('sample-c')
ParseFileData('sample-d')
ParseFileData('sample-e')
ParseFileData('sample-f')
ParseFileData('sample-g')


In [68]:
sc.stop()